In [93]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from pprint import pprint
from scipy.stats import ttest_ind
from datetime import datetime
from config import yelp_key

In [2]:
county_datafile = "data/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv"

county_data = pd.read_csv(county_datafile)
county_data.count()

ACTIVITY DATE           199950
OWNER ID                199950
OWNER NAME              199950
FACILITY ID             199950
FACILITY NAME           199950
RECORD ID               199950
PROGRAM NAME            199950
PROGRAM STATUS          199950
PROGRAM ELEMENT (PE)    199950
PE DESCRIPTION          199950
FACILITY ADDRESS        199950
FACILITY CITY           199950
FACILITY STATE          199950
FACILITY ZIP            199950
SERVICE CODE            199950
SERVICE DESCRIPTION     199950
SCORE                   199950
GRADE                   199895
SERIAL NUMBER           199950
EMPLOYEE ID             199950
dtype: int64

In [16]:
columns = ['ACTIVITY DATE','FACILITY ID','FACILITY NAME','PROGRAM STATUS','FACILITY ADDRESS','FACILITY CITY','FACILITY ZIP','SCORE','GRADE']
cleaned_county_data = county_data[columns]
cleaned_county_data['ACTIVITY DATE'] = pd.to_datetime(cleaned_county_data['ACTIVITY DATE'])
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
0,2016-06-22,FA0179753,TEA BAR,ACTIVE,954 E ALOSTA AVE,AZUSA,91702,93,A
1,2016-09-07,FA0239319,SHEFA,ACTIVE,7275 MELROSE AVE,LOS ANGELES,90046,90,A
2,2016-09-09,FA0071154,SUBWAY,ACTIVE,28788 THE OLD RD,VALENCIA,91355,98,A
3,2016-04-01,FA0144494,AFC SUSHI @ VONS #3138,ACTIVE,16550 SOLEDAD CANYON RD,CANYON COUNTRY,91387-3215,99,A
4,2016-04-01,FA0005693,ALBERTS MEXICAN FOOD,INACTIVE,2355 PECK RD,SOUTH EL MONTE,91733,92,A


In [17]:
cleaned_county_data.sort_values(by = ["ACTIVITY DATE","FACILITY ID","FACILITY ADDRESS"],inplace = True,ascending=True)
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
94,2016-04-01,FA0002213,RANCHITO MILLAN,ACTIVE,5803 S SOTO ST,HUNTINGTON PARK,90255,94,A
12,2016-04-01,FA0004764,BALDWIN OIL,ACTIVE,13758 LOS ANGELES ST,BALDWIN PARK,91706,98,A
14,2016-04-01,FA0004876,BANQUET KITCHEN,ACTIVE,135 N GRAND AVE,LOS ANGELES,90012,97,A
5,2016-04-01,FA0005575,ALL STAR PARK COMPANY,INACTIVE,17911 S BELLFLOWER BLVD,BELLFLOWER,90706,91,A
17,2016-04-01,FA0005633,BIG O LIQUIOR,INACTIVE,4626 S VERMONT AVE,LOS ANGELES,90037,95,A


In [18]:
unique_county_data = cleaned_county_data.drop_duplicates(subset = ['FACILITY ID'], keep = 'last')
unique_county_data.count()

ACTIVITY DATE       44581
FACILITY ID         44581
FACILITY NAME       44581
PROGRAM STATUS      44581
FACILITY ADDRESS    44581
FACILITY CITY       44581
FACILITY ZIP        44581
SCORE               44581
GRADE               44563
dtype: int64

In [19]:
unique_county_data = unique_county_data.loc[(unique_county_data['PROGRAM STATUS'] != "INACTIVE"),:]
unique_county_data.count()

ACTIVITY DATE       36474
FACILITY ID         36474
FACILITY NAME       36474
PROGRAM STATUS      36474
FACILITY ADDRESS    36474
FACILITY CITY       36474
FACILITY ZIP        36474
SCORE               36474
GRADE               36462
dtype: int64

In [20]:
unique_county_data = unique_county_data.dropna(how='any')
unique_county_data.count()

ACTIVITY DATE       36462
FACILITY ID         36462
FACILITY NAME       36462
PROGRAM STATUS      36462
FACILITY ADDRESS    36462
FACILITY CITY       36462
FACILITY ZIP        36462
SCORE               36462
GRADE               36462
dtype: int64

In [69]:
unique_county_data.head(10)

,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
540,2016-04-05,FA0034994,LA SPICE,ACTIVE,4357 SEPULVEDA BLVD,CULVER CITY,90230,91,A
3918,2016-04-20,FA0064275,JEAN'S MARKET,ACTIVE,919 N FOREST AVE,LOS ANGELES,90033,95,A
4251,2016-04-21,FA0159729,BAR LATINO,ACTIVE,4675 MELROSE AVE,LOS ANGELES,90029,96,A
6489,2016-05-02,FA0006151,AMERICAN LEGION #465,ACTIVE,7550 EASTERN AVE,BELL GARDENS,90201,97,A
8434,2016-05-10,FA0021740,LEE'S ARCO,ACTIVE,5804 S CRENSHAW BLVD,LOS ANGELES,90043,96,A
8438,2016-05-10,FA0031590,LLENO DE VIDA,ACTIVE,13046 GLENOAKS BLVD,SYLMAR,91342,93,A
8545,2016-05-10,FA0048259,SHAMROCK LOUNGE,ACTIVE,7400 STATE ST,HUNTINGTON PARK,90255,95,A
8583,2016-05-10,FA0071811,TAHI BBQ RESTAURANT,ACTIVE,851 S VERMONT AVE,LOS ANGELES,90005,82,B
9259,2016-05-12,FA0151120,LA FAMILY HOUSING,ACTIVE,7843 LANKERSHIM BLVD,NORTH HOLLYWOOD,91605-2523,99,A
10638,2016-05-18,FA0010200,CRAZY GIRLS,ACTIVE,1433 N LA BREA AVE,LOS ANGELES,90028,90,A


In [53]:
print(unique_county_data['FACILITY ZIP'].iloc[0])
print(unique_county_data['FACILITY NAME'].iloc[0])

90230
LA SPICE


In [67]:
#How to get a subset of the dataframe
#unique_county_data[unique_county_data['FACILITY CITY'] == 'SANTA MONICA']
#subset_df = unique_county_data.iloc[10:21,:]

In [59]:
headers = {"Authorization": "Bearer "+yelp_key}
params = {"term": unique_county_data['FACILITY NAME'].iloc[0],"location": unique_county_data['FACILITY ZIP'].iloc[0]}
target_url = "https://api.yelp.com/v3/businesses/search"

yelp_reviews = requests.get(target_url, params, headers=headers).json()
yelp_reviews

{'businesses': [{'id': 'pYCLjAlGO-xRb3Ia_-GjHg',
   'alias': 'la-spice-culver-city-3',
   'name': 'LA Spice',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/EiJUNS1PMVQMS822vMJOSw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/la-spice-culver-city-3?adjust_creative=XjN-ow4TKOIVxoXsGokQhA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=XjN-ow4TKOIVxoXsGokQhA',
   'review_count': 17,
   'categories': [{'alias': 'catering', 'title': 'Caterers'},
    {'alias': 'eventplanning', 'title': 'Party & Event Planning'}],
   'rating': 5.0,
   'coordinates': {'latitude': 34.003373, 'longitude': -118.408153},
   'transactions': [],
   'location': {'address1': '4357 Sepulveda Blvd',
    'address2': '',
    'address3': '',
    'city': 'Culver City',
    'zip_code': '90230',
    'country': 'US',
    'state': 'CA',
    'display_address': ['4357 Sepulveda Blvd', 'Culver City, CA 90230']},
   'phone': '+14245002125',
   'display_phone': '(424) 500-2125',


In [113]:
headers = {"Authorization": "Bearer "+yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

rating = []
review_count = []
price = []

for index, row in unique_county_data.iloc[:10,:].iterrows():
    restr_name = row['FACILITY NAME']
    restr_zip = row['FACILITY ZIP']
    restr_addr = row['FACILITY ADDRESS']
    restr_city = row['FACILITY CITY']
    params = {"term": restr_name,"location": str(restr_loc + ", " + restr_city + ", "+restr_zip)}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        print(restr_name)
        try:
            rating.append(yelp_data['businesses'][0]['rating'])
            print(yelp_data['businesses'][0]['rating'])
        except (KeyError, IndexError):
            print("Missing Name...")
        try:
            review_count.append(yelp_data['businesses'][0]['review_count'])
            print(yelp_data['businesses'][0]['review_count'])
        except (KeyError, IndexError):
            print("Missing Reviews...")    
        try:    
            review_count.append(yelp_data['businesses'][0]['price'])
            print(yelp_data['businesses'][0]['price'])
        except (KeyError, IndexError):
            print("Missing Price...")


LA SPICE
5.0
17
Missing Price...
JEAN'S MARKET
5.0
4
$$
BAR LATINO
4.5
4
Missing Price...
AMERICAN LEGION #465
Missing Name...
Missing Reviews...
Missing Price...
LEE'S ARCO
5.0
1
Missing Price...
LLENO DE VIDA
2.0
1
Missing Price...
SHAMROCK LOUNGE
4.0
523
$$
TAHI BBQ RESTAURANT
4.0
797
$$
LA FAMILY HOUSING
5.0
1
Missing Price...
CRAZY GIRLS
3.5
95
$$


In [89]:
rating

[5.0]

In [102]:
pprint(yelp_data['businesses'][3])

TypeError: list indices must be integers or slices, not tuple

In [97]:
yelp_data['businesses'][1]['price']

'$'